# Working splits

In [164]:
myobs = "abcdefghijklmnopqrstuvwxyz"

L = 5
for k in range(0, 2*L-1 + 1):
    A0 = L - k
    A1 = L
    if A0 <= 0:
        A1 = L + A0 - 1
        A0 = 1
    B0 = A1
    B1 = A0

    print(f"k = {k} => total = {2*L + 1 - k}")
    print("2-splits:", end=' ')
    for A, B in zip(range(A0, A1+1), range(B0, B1 - 1, -1)):
        xj = myobs[0 : A]
        z = myobs[0 + A]
        xi = myobs[0 + A+1 : 0 + A+1 + B]
        print(f"{xj}/{z}/{xi}", end=', ')
    
    C0 = L - (k-1)
    C1 = L
    if C0 <= 0:
        C1 = L + C0
        C0 = 1
    D0 = C1
    D1 = C0
    print()
    print("1-splits:", end=' ')
    for C, D in zip(range(C0, C1+1), range(D0, D1 - 1, -1)):
        print(f"{C}/{D}", end=', ')
    
    print('\n')

k = 0 => total = 11
2-splits: abcde/f/ghijk, 
1-splits: 

k = 1 => total = 10
2-splits: abcd/e/fghij, abcde/f/ghij, 
1-splits: 5/5, 

k = 2 => total = 9
2-splits: abc/d/efghi, abcd/e/fghi, abcde/f/ghi, 
1-splits: 4/5, 5/4, 

k = 3 => total = 8
2-splits: ab/c/defgh, abc/d/efgh, abcd/e/fgh, abcde/f/gh, 
1-splits: 3/5, 4/4, 5/3, 

k = 4 => total = 7
2-splits: a/b/cdefg, ab/c/defg, abc/d/efg, abcd/e/fg, abcde/f/g, 
1-splits: 2/5, 3/4, 4/3, 5/2, 

k = 5 => total = 6
2-splits: a/b/cdef, ab/c/def, abc/d/ef, abcd/e/f, 
1-splits: 1/5, 2/4, 3/3, 4/2, 5/1, 

k = 6 => total = 5
2-splits: a/b/cde, ab/c/de, abc/d/e, 
1-splits: 1/5, 2/4, 3/3, 4/2, 5/1, 

k = 7 => total = 4
2-splits: a/b/cd, ab/c/d, 
1-splits: 1/4, 2/3, 3/2, 4/1, 

k = 8 => total = 3
2-splits: a/b/c, 
1-splits: 1/3, 2/2, 3/1, 

k = 9 => total = 2
2-splits: 
1-splits: 1/2, 2/1, 



# Working indexing

In [138]:
import pandas as pd

In [161]:
x = pd.DataFrame(0.0, columns = [0], index = [0], dtype=float)

N = 1000

xjxis = [("cdf", "dfi"), ("dfb", "aba"), ("bbd", "aba"), ("cdf", "dfi"), ("dfb", "aba")] 

for xj, xi in xjxis:
    if xj not in x.columns:
        x.loc[:, xj] = 0.0
        print(f"Added {xj} to x columns")
    if xi not in x.index:
        x.loc[xi, :] = 0.0
        print(f"Added {xi} to x rows")
    x.loc[xi, xj] += 1 / (N - len(xj) - len(xi) + 1)
    print(x, '\n')

x.drop(0, axis=0, inplace=True)
x.drop(0, axis=1, inplace=True)
x.values

Added cdf to x columns
Added dfi to x rows
       0       cdf
0    0.0  0.000000
dfi  0.0  0.001005 

Added dfb to x columns
Added aba to x rows
       0       cdf       dfb
0    0.0  0.000000  0.000000
dfi  0.0  0.001005  0.000000
aba  0.0  0.000000  0.001005 

Added bbd to x columns
       0       cdf       dfb       bbd
0    0.0  0.000000  0.000000  0.000000
dfi  0.0  0.001005  0.000000  0.000000
aba  0.0  0.000000  0.001005  0.001005 

       0      cdf       dfb       bbd
0    0.0  0.00000  0.000000  0.000000
dfi  0.0  0.00201  0.000000  0.000000
aba  0.0  0.00000  0.001005  0.001005 

       0      cdf      dfb       bbd
0    0.0  0.00000  0.00000  0.000000
dfi  0.0  0.00201  0.00000  0.000000
aba  0.0  0.00000  0.00201  0.001005 



array([[0.00201005, 0.        , 0.        ],
       [0.        , 0.00201005, 0.00100503]])

# Combining them

In [363]:
def reduce_to_common(matrices: list[pd.DataFrame], series_i0, series_0j, max_length):
    index_set = set(series_i0.index)
    column_set = set(series_0j.index)

    for matrix in matrices:
        index_set &= set(matrix.index)
        column_set &= set(matrix.columns)
    
    for matrix in matrices:
        # # Drop 0-row and 0-column
        # matrix.drop(0.0, axis=0, inplace=True)
        # matrix.drop(0.0, axis=1, inplace=True)
        
        # Drop uncommon rows
        matrix_index = set(matrix.index)
        to_drop = matrix_index - index_set
        matrix.drop(to_drop, axis=0, inplace=True)

        # Drop uncommon columns
        matrix_columns = set(matrix.columns)
        to_drop = matrix_columns - column_set
        matrix.drop(to_drop, axis=1, inplace=True)

        # Reorder indices alphabetically and by word length
        matrix.sort_index(axis=1, key=lambda x: x.str.rjust(max_length, 'Z'), inplace=True)
        matrix.sort_index(axis=0, key=lambda x: x.str.rjust(max_length, 'Z'), inplace=True)

    series_index = set(series_i0.index)
    to_drop = series_index - index_set
    series_i0.drop(to_drop, inplace=True)
    series_i0.sort_index(key=lambda x: x.str.rjust(max_length, 'Z'), inplace=True)

    series_index = set(series_0j.index)
    to_drop = series_index - index_set
    series_0j.drop(to_drop, inplace=True)
    series_0j.sort_index(key=lambda x: x.str.rjust(max_length, 'Z'), inplace=True)


def get_matrices(myobs, alphabet, max_length):
    N = len(myobs)
    L = max_length
    
    F_IzJ = dict(
        zip(
            [0] + alphabet,
            [{} for _ in range(len(alphabet) + 1)]
        )
    )
    F_I0 = {}
    F_0J = {}
    
    for k in range(0, 2*L-1 + 1):
        print(k, end = ' ')
        
        # Min/max ranges for 2-splits
        A0 = L - k
        A1 = L
        if A0 <= 0:
            A1 = L + A0 - 1
            A0 = 1
        B0 = A1
        B1 = A0

        # Min/max ranges for 1-splits
        C0 = L - (k-1)
        C1 = L
        if C0 <= 0:
            C1 = L + C0
            C0 = 1
        D0 = C1
        D1 = C0
        
        for start in range(0, N - (2*L + 1 - k)):
            
            for A, B in zip(range(A0, A1+1), range(B0, B1 - 1, -1)):
                # Get each 2-split
                xj = myobs[start : start + A]
                z = myobs[start + A]
                xi = myobs[start + A+1 : start + A+1 + B]
                # print(len(xj+z+xi))
    
                # Add to F_IzJ = F_IzJ[z]
                if xi not in F_IzJ[z]:
                    F_IzJ[z][xi] = {}
                if xj not in F_IzJ[z][xi]:
                    F_IzJ[z][xi][xj] = 0.0
                F_IzJ[z][xi][xj] += 1 / (N - len(xj) - len(xi) + 1)
            
            for C, D in zip(range(C0, C1+1), range(D0, D1 - 1, -1)):
                # Get each 1-split
                xj = myobs[start : start + C]
                xi = myobs[start + C : start + C + D]
        
                # Add to F_IJ = F_IzJ[0]
                if xi not in F_IzJ[0]:
                    F_IzJ[0][xi] = {}
                if xj not in F_IzJ[0][xi]:
                    F_IzJ[0][xi][xj] = 0.0
                F_IzJ[0][xi][xj] += 1 / (N - len(xj) - len(xi) + 1)

                # Add to F_0J
                if xj not in F_0J:
                    F_0J[xj] = 0.0
                F_0J[xj] += 1 / (N - len(xj) + 1)

                # Add to F_I0
                if xi not in F_I0:
                    F_I0[xi] = 0.0
                F_I0[xi] += 1 / (N - len(xi) + 1)
    
            # Get each characteristic word

    for key, entry in F_IzJ.items():
        F_IzJ[key] = pd.DataFrame.from_dict(
            entry, orient='index', dtype=float
        ).fillna(0)
    
    F_I0 = pd.Series(F_I0)
    F_0J = pd.Series(F_0J)
    
    reduce_to_common(list(F_IzJ.values()), F_I0, F_0J, L)

    for obs, matrix in F_IzJ.items():
        F_IzJ[obs] = np.asmatrix(matrix.values)
    F_I0 = np.asmatrix(F_I0.values)
    F_0J = np.asmatrix(F_0J.values).T
    
    return F_IzJ, F_0J, F_I0

In [364]:
myobs = "aaabbbaaabbbaaabbbabbababbabababaabbabab"
alphabet = ["a", "b"]
L = 3

mats, f_i0, f_0j = get_matrices(myobs, alphabet, L)

0 1 2 3 4 5 

In [371]:
np.set_printoptions(edgeitems=30, linewidth=200)#, formatter=dict(float=lambda x: "%.3g" % x))
mats

{0: matrix([[0.        , 0.        , 0.07894737, 0.13157895, 0.07894737, 0.15789474, 0.05405405, 0.16216216, 0.10810811, 0.27027027, 0.10810811],
         [0.        , 0.        , 0.10526316, 0.15789474, 0.21052632, 0.07894737, 0.18918919, 0.16216216, 0.05405405, 0.10810811, 0.21621622],
         [0.07894737, 0.07894737, 0.        , 0.05405405, 0.10810811, 0.10810811, 0.        , 0.        , 0.        , 0.02777778, 0.05555556],
         [0.10526316, 0.21052632, 0.16216216, 0.18918919, 0.05405405, 0.21621622, 0.02777778, 0.08333333, 0.05555556, 0.08333333, 0.        ],
         [0.13157895, 0.15789474, 0.        , 0.16216216, 0.27027027, 0.16216216, 0.05555556, 0.08333333, 0.        , 0.05555556, 0.08333333],
         [0.15789474, 0.07894737, 0.21621622, 0.16216216, 0.10810811, 0.        , 0.02777778, 0.        , 0.02777778, 0.        , 0.02777778],
         [0.16216216, 0.05405405, 0.        , 0.02777778, 0.05555556, 0.        , 0.        , 0.        , 0.        , 0.02857143, 0.0571428

In [345]:
mytest = {
    "a": {
        "ab": 0.3,
        "a": 0.1,
        "bba": 0.01
    },
    "aa": {
        "a": 0.05,
        "ab": 0.1,
        "b": 0.1
    },
    "ba": {
        "bab": 0.02,
        "aa": 0.1,
        "bba": 0.1,
        "a": 0.05
    },
    "bba": {
        "a": 0.1
    }
}

In [352]:
pd.DataFrame.from_dict(mytest, orient='index', dtype=float).fillna(0)

,ab,a,bba,b,bab,aa
a,0.3,0.10,0.01,0.0,0.00,0.0
aa,0.1,0.05,0.00,0.1,0.00,0.0
ba,0.0,0.05,0.10,0.0,0.02,0.1
bba,0.0,0.10,0.00,0.0,0.00,0.0
